In [1]:
!pip install opendatasets datasets transformers datasets peft accelerate bitsandbytes --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00


In [2]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/gowrishankarp/newspaper-text-summarization-cnn-dailymail")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: wadiaatouhami
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/gowrishankarp/newspaper-text-summarization-cnn-dailymail


100%|██████████| 503M/503M [00:08<00:00, 65.2MB/s]


In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
    GenerationConfig, TrainingArguments, Trainer
)
from peft import LoraConfig, get_peft_model
import pandas as pd
from datasets import Dataset
import re

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
train_df = pd.read_csv("/content/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv")[["article", "highlights"]]
data_df = train_df.sample(5000)
data_df

,article,highlights
9789,By . Stephen Mcgowan for MailOnline . Rangers ...,Ex-Rangers commercial director Imran Ahmad goe...
90845,(CNN) -- Who doesn't love a good holiday-theme...,"""The Mindy Project"" will kick off prime time's..."
196966,"By . Ruth Styles . PUBLISHED: . 12:55 EST, 7 J...",Types include 'sharks' who are too cool to set...
139002,A group of Scottish football legends have come...,Former Match of the Day pundit Alan Hansen has...
286357,Never-before-released Interviews with veteran ...,"Interviews with female CIA employees, who join..."
...,...,...
58327,(CNN) -- There was lots of news in the NFL thi...,Senator: NFL commissioner should resign if the...
233187,By . Daily Mail Reporter . PUBLISHED: . 17:39 ...,Lonigan's in Estes Park is next to Rocky Mount...
64969,"By . Ryan Gorman . PUBLISHED: . 21:34 EST, 26 ...","Sir Richard Branson was in town, sources told ..."
33681,Says Millie Mackintosh is 'queenie' these days...,Says Millie Mackintosh is 'queenie' these days...


In [5]:
def filter_text(text):
  text = text.lower()
  text = re.sub('[^A-Za-z0-9]+', ' ', text)
  return text

data_df["article"] = data_df["article"].apply(filter_text)
data_df["highlights"] = data_df["highlights"].apply(filter_text)
data_df

,article,highlights
9789,by stephen mcgowan for mailonline rangers edge...,ex rangers commercial director imran ahmad goe...
90845,cnn who doesn t love a good holiday themed ep...,the mindy project will kick off prime time s ...
196966,by ruth styles published 12 55 est 7 june 2013...,types include sharks who are too cool to settl...
139002,a group of scottish football legends have come...,former match of the day pundit alan hansen has...
286357,never before released interviews with veteran ...,interviews with female cia employees who joine...
...,...,...
58327,cnn there was lots of news in the nfl this we...,senator nfl commissioner should resign if ther...
233187,by daily mail reporter published 17 39 est 25 ...,lonigan s in estes park is next to rocky mount...
64969,by ryan gorman published 21 34 est 26 august 2...,sir richard branson was in town sources told m...
33681,says millie mackintosh is queenie these days m...,says millie mackintosh is queenie these days m...


In [6]:
data_df["final_statement"] = "Summarize the following article.\n\n" + data_df["article"] + "\Summary:\n" + data_df["highlights"]
data_df.sample(1)

,article,highlights,final_statement
277198,cnn in the wake of the tragic shootings at co...,10 years after columbine schools are taking a ...,Summarize the following article.\n\n cnn in th...


In [7]:
data_df

,article,highlights,final_statement
9789,by stephen mcgowan for mailonline rangers edge...,ex rangers commercial director imran ahmad goe...,Summarize the following article.\n\nby stephen...
90845,cnn who doesn t love a good holiday themed ep...,the mindy project will kick off prime time s ...,Summarize the following article.\n\n cnn who d...
196966,by ruth styles published 12 55 est 7 june 2013...,types include sharks who are too cool to settl...,Summarize the following article.\n\nby ruth st...
139002,a group of scottish football legends have come...,former match of the day pundit alan hansen has...,Summarize the following article.\n\na group of...
286357,never before released interviews with veteran ...,interviews with female cia employees who joine...,Summarize the following article.\n\nnever befo...
...,...,...,...
58327,cnn there was lots of news in the nfl this we...,senator nfl commissioner should resign if ther...,Summarize the following article.\n\n cnn there...
233187,by daily mail reporter published 17 39 est 25 ...,lonigan s in estes park is next to rocky mount...,Summarize the following article.\n\nby daily m...
64969,by ryan gorman published 21 34 est 26 august 2...,sir richard branson was in town sources told m...,Summarize the following article.\n\nby ryan go...
33681,says millie mackintosh is queenie these days m...,says millie mackintosh is queenie these days m...,Summarize the following article.\n\nsays milli...


In [8]:
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-1b1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [9]:
def tokenize_function(example):
    example["input_ids"] = tokenizer(example["final_statement"], padding="max_length", max_length = 300, truncation=True, return_tensors="pt").input_ids
    example["labels"] = tokenizer(example["final_statement"], padding="max_length", max_length = 300, truncation=True, return_tensors="pt").input_ids
    return example

In [10]:
train = data_df.sample(frac=0.8,random_state=7)
valid = data_df.drop(train.index)

In [11]:
print("Training Shape: ", train.shape)
print("Validation Shape: ", valid.shape)

Training Shape:  (4000, 3)
Validation Shape:  (1000, 3)


In [12]:
# Convert your DataFrame into a Dataset object
train_dataset = Dataset.from_pandas(train)
valid_dataset = Dataset.from_pandas(valid)

# Apply the tokenize function
train_tokenized_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=train_dataset.column_names)
valid_tokenized_dataset = valid_dataset.map(tokenize_function, batched=True, remove_columns=valid_dataset.column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [13]:
train_tokenized_dataset

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 4000
})

In [14]:
valid_tokenized_dataset

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1000
})

In [15]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
)

model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-1b1", quantization_config=quant_config)

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/2.13G [00:00<?, ?B/s]

In [16]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM",
)

peft_model = get_peft_model(model, peft_params)

In [17]:
peft_model.print_trainable_parameters()

trainable params: 1,179,648 || all params: 1,066,493,952 || trainable%: 0.1106


In [18]:
training_args = TrainingArguments(
    output_dir='./model_checkpoints',
    save_total_limit=1,
    auto_find_batch_size=True,
    learning_rate=1e-3,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    eval_steps=500,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
# create trainer object
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_tokenized_dataset,
    eval_dataset=valid_tokenized_dataset
)

# train model
trainer.train()
trainer.model.save_pretrained('./final_model')
tokenizer.save_pretrained('./final_model')

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.364800,3.341797
2,3.235900,3.333984
3,3.129200,3.341797


('./final_model/tokenizer_config.json',
 './final_model/special_tokens_map.json',
 './final_model/tokenizer.json')

In [20]:
final_model = AutoModelForCausalLM.from_pretrained("final_model")
final_tokenizer = AutoTokenizer.from_pretrained("final_model")

In [25]:
news_article = """
All but one of the 100 cities with the world’s worst air pollution last year were in Asia, according to a new report, with the climate crisis playing a pivotal role in bad air quality that is risking the health of billions of people worldwide.

The vast majority of these cities — 83 — were in India and all exceeded the World Health Organization’s air quality guidelines by more than 10 times, according to the report by IQAir, which tracks air quality worldwide.

The study looked specifically at fine particulate matter, or PM2.5, which is the tiniest pollutant but also the most dangerous. Only 9% of more than 7,800 cities analyzed globally recorded air quality that met WHO’s standard, which says average annual levels of PM2.5 should not exceed 5 micrograms per cubic meter.

“We see that in every part of our lives that air pollution has an impact,” said IQAir Global CEO Frank Hammes. “And it typically, in some of the most polluted countries, is likely shaving off anywhere between three to six years of people’s lives. And then before that will lead to many years of suffering that are entirely preventable if there’s better air quality.”

"""
full_prompt = "Summarize the following article.\n\n" + filter_text(news_article) + "\nSummary:\n"

input_ids = final_tokenizer(full_prompt, padding="max_length", max_length = 300, truncation=True, return_tensors="pt").input_ids

In [26]:
output = final_model.generate(input_ids, max_new_tokens = 100)
summary = final_tokenizer.decode(output[0], skip_special_tokens = True)

In [27]:
print(summary)

Summarize the following article.

 all but one of the 100 cities with the world s worst air pollution last year were in asia according to a new report with the climate crisis playing a pivotal role in bad air quality that is risking the health of billions of people worldwide the vast majority of these cities 83 were in india and all exceeded the world health organization s air quality guidelines by more than 10 times according to the report by iqair which tracks air quality worldwide the study looked specifically at fine particulate matter or pm2 5 which is the tiniest pollutant but also the most dangerous only 9 of more than 7 800 cities analyzed globally recorded air quality that met who s standard which says average annual levels of pm2 5 should not exceed 5 micrograms per cubic meter we see that in every part of our lives that air pollution has an impact said iqair global ceo frank hammes and it typically in some of the most polluted countries is likely shaving off anywhere between

In [28]:
print(summary.split("\nSummary:\n")[1])

the study looked specifically at fine particulate matter or pm2 5 which is the tiniest pollutant but also the most dangerous only 9 of more than 7 800 cities analyzed globally recorded air quality that met who s standard which says average annual levels of pm2 5 should not exceed 5 micrograms per cubic meter we see that in every part of our lives that air pollution has an impact the report looked specifically at fine particulate matter or pm2 5 which is the tiniest pollut
